In [8]:
track_runs_sheet_path = "runs_sheet.json"
non_passing_instances_path = "/Users/zi/codestory/SWE-bench/eval_scripts/instance_ids_django_others.txt"

In [9]:
# open file
with open(non_passing_instances_path, "r") as file:
    non_passing_instances = file.readlines()

total_instances = 500
non_passing_instances_count = len(non_passing_instances)
passing_instances_count = total_instances - non_passing_instances_count

print("Non passing instances: ", non_passing_instances_count)
print("Passing instances: ", passing_instances_count)
print("Pass %: ", passing_instances_count / total_instances * 100)

Non passing instances:  391
Passing instances:  109
Pass %:  21.8


In [10]:
# Open our sheet
import json

with open(track_runs_sheet_path, "r") as file:
    runs_sheet = json.load(file)

log_sheet_id, sheet_id, sheet_name = runs_sheet["LOG_SHEET_ID"], runs_sheet["SHEET_ID"], runs_sheet["LOG_SHEET_NAME"]

if log_sheet_id is None or sheet_id is None or sheet_name is None:
    raise Exception("Sheet ID or name is None")


1W0gxh-NC9Sl01yrlTRPNGvyDQva3_lZPPPMQ2M8IP74 280623479 RUNS


In [11]:
# load sheet, specifically column with name "OVERALL"
from swebench.google_sheets import get_column_values

overall_column_name = "OVERALL"
overall_column_values = get_column_values(log_sheet_id, sheet_name, overall_column_name)

print(overall_column_values)

/Users/zi/codestory/SWE-bench/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-10 12:19:28,291 - googleapiclient.discovery_cache - INFO - file_cache is only supported with oauth2client<4.0.0


['TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'FAL